In [1]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K


/Users/jiayun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jiayun/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.2

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())



In [3]:
# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# model.summary()

Model loaded.


In [4]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_pool'

In [5]:
import pandas as pd

coef = pd.read_csv('model.csv')
coef = coef.drop(coef.columns[[0,1]], axis=1)
coef = coef.as_matrix()

In [6]:
fil = pd.read_csv('filter.csv')
fil = fil.drop(fil.columns[0], axis=1)
fil = fil.values

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_input = mpimg.imread('/Users/jiayun/Documents/coding/vgg/data/dog128_2.jpg')
# print(img_input.shape)
img_input = np.reshape(img_input, (1,img_width,img_height,3))

In [14]:
# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

# store result
kept_filters = []
id_filters = []

for model_index in range(4):
    # we visualize 4 models
    print('Processing model %d' % model_index)
    start_time = time.time()
    
    # read model
    thisModel = coef[model_index]
    
    # we build a loss function that maximizes the activation of the nth model
    layer_output = layer_dict[layer_name].output
    loss = 0 # initialize
    for filter_index in range(len(fil)):
        loss += thisModel[filter_index] * K.mean(layer_output[:, :, :, filter_index])  # "-=" more like a cat and "+=" more like a dog

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 5

    # we start from a gray image with some random noise
    input_img_data = img_input
    input_img_data = (input_img_data-0.0)

    # we run gradient ascent for 40 steps
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        if i % 10 == 0:
            print('Current loss value:', loss_value)

    print('Current loss value:', loss_value)
    
    img = deprocess_image(input_img_data[0])
    kept_filters.append((img, loss_value))
    id_filters.append(filter_index)
    
    end_time = time.time()
    print('model %d processed in %ds' % (model_index, end_time - start_time))


Processing model 0
Current loss value: 3.5902905
Current loss value: 3041.2837
Current loss value: 7314.605
Current loss value: 12213.682
Current loss value: 17170.094
model 0 processed in 15s
Processing model 1
Current loss value: -3.6865754
Current loss value: 2954.8418
Current loss value: 7442.9756
Current loss value: 12659.539
Current loss value: 17887.756
model 1 processed in 17s
Processing model 2
Current loss value: -9.780815
Current loss value: 3333.534
Current loss value: 7773.305
Current loss value: 12799.64
Current loss value: 17782.945
model 2 processed in 15s
Processing model 3
Current loss value: 9.338768
Current loss value: 3378.3467
Current loss value: 8376.837
Current loss value: 13843.301
Current loss value: 19400.018
model 3 processed in 14s


In [15]:
print(len(kept_filters))

4


In [16]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 2
# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
#kept_filters.sort(key=lambda x: x[1], reverse=True)
#kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('activation.png', stitched_filters)

In [17]:
input_img_data = img_input
input_img_data = (input_img_data-0.0)
img_original = deprocess_image(input_img_data[0])

In [18]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 2
# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
#kept_filters.sort(key=lambda x: x[1], reverse=True)
#kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        img = abs(img - img_original)
#         img = img/(img.max() + 0.001)
#         img *= 255
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('activation2.png', stitched_filters)

In [19]:
from PIL import Image
img = Image.open('activation2.png').convert('LA')
img.save('activation2.png')

In [24]:
# # compare with input_img_data

# # we will stich the best 64 filters on a 8 x 8 grid.
# n = 2
# # the filters that have the highest loss are assumed to be better-looking.
# # we will only keep the top 64 filters.
# #kept_filters.sort(key=lambda x: x[1], reverse=True)
# #kept_filters = kept_filters[:n * n]

# # build a black picture with enough space for
# # our 8 x 8 filters of size 128 x 128, with a 5px margin in between
# margin = 5
# width = n * img_width + (n - 1) * margin
# height = n * img_height + (n - 1) * margin
# stitched_filters = np.zeros((width, height))

# # fill the picture with our saved filters
# for i in range(n):
#     for j in range(n):
#         img, loss = kept_filters[i * n + j]
#         img = abs(img - img_original)
#         Y = 0.299 * img[:,:,0] + 0.587 * img[:,:,1] + 0.114 * img[:,:,2]
#         width_margin = (img_width + margin) * i
#         height_margin = (img_height + margin) * j
#         stitched_filters[
#             width_margin: width_margin + img_width,
#             height_margin: height_margin + img_height] = Y

# # save the result to disk
# plt.imshow(stitched_filters, cmap = plt.get_cmap('gray'))
# plt.show()
# plt.savefig('activation3.jpg')